In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

# To install fairseq from source and develop locally:
!git clone https://github.com/choyi0521/fairseq
%cd /content/fairseq
!pip install --editable .

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into 'fairseq'...
remote: Enumerating objects: 12181, done.
remote: Total 12181 (delta 0), reused 0 (delta 0), pack-reused 12181
Receiving objects: 100% (12181/12181), 5.71 MiB | 6.90 MiB/s, done.
Resolving deltas: 100% (8965/8965), done.
/content/fairseq
Obtaining file:///content/fairseq
     |████████████████████████████████| 645kB 9.0MB/s 
  Running setup.py develop for fairseq


In [2]:
!git checkout dev6
!git pull origin dev6

Branch 'dev6' set up to track remote branch 'dev6' from 'origin'.
Switched to a new branch 'dev6'
From https://github.com/choyi0521/fairseq
 * branch            dev6       -> FETCH_HEAD
Already up to date.


In [3]:
# Download and prepare the data
%cd /content/fairseq
%cd examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

# Preprocess/binarize the data
TEXT="examples/translation/iwslt14.tokenized.de-en"
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt14.tokenized.de-en \
    --workers 20

/content/fairseq
/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 147464 (delta 104), reused 118 (delta 74), pack-reused 147295
Receiving objects: 100% (147464/147464), 129.73 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (113938/113938), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 538 (delta 10), reused 21 (delta 7), pack-reused 509
Receiving objects: 100% (538/538), 226.98 KiB | 873.00 KiB/s, done.
Resolving deltas: 100% (316/316), done.
--2019-12-05 08:28:02--  https://wit3.fbk.eu/archive/2014-01/texts/de/en/de-en.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 

In [0]:
SAVE="/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt"

In [5]:
# Training
%cd /content/fairseq
!mkdir -p "{SAVE}" 
!CUDA_VISIBLE_DEVICES=0 $(which fairseq-train) data-bin/iwslt14.tokenized.de-en \
    --encoder-conv-type ddynamic --decoder-conv-type ddynamic \
    --clip-norm 0 --optimizer adam --lr 0.0005 \
    --source-lang de --target-lang en --max-tokens 4000 --no-progress-bar \
    --log-interval 100 --min-lr '1e-09' --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --lr-scheduler inverse_sqrt \
    --ddp-backend=no_c10d \
    --max-update 80000 --warmup-updates 4000 --warmup-init-lr '1e-07' \
    --adam-betas '(0.9, 0.98)' \
    -a lightconv_iwslt_de_en --save-dir "{SAVE}" \
    --dropout 0.3 --attention-dropout 0.1 --weight-dropout 0.1 \
    --encoder-glu 0 --decoder-glu 0 \
    --encoder-gau 0 --decoder-gau 0 \
    --conv-mixed 0 \
    --encoder-conv-dim 512 --decoder-conv-dim 512 \
    --encoder-query-dim 512 --decoder-query-dim 512 \
    --encoder-attention-heads 4 --decoder-attention-heads 4 \
    --encoder-attention-query-heads 1 --decoder-attention-query-heads 1 \
    2>&1 | tee "{SAVE}/train3.log"

/content/fairseq
Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='lightconv_iwslt_de_en', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, conv_mixed=False, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data-bin/iwslt14.tokenized.de-en', dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=4, decoder_attention_query_heads=1, decoder_conv_dim=512, decoder_conv_type='ddynamic', decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_gau=False, decoder_glu=False, decoder_input_dim=512, decoder_kernel_size_list=[3, 7, 15, 31, 31, 31], decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, decoder_query_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distribute

In [6]:
!python scripts/average_checkpoints.py --inputs "{SAVE}" \
    --num-epoch-checkpoints 10 --output "{SAVE}/checkpoint_last10_avg.pt"

# Evaluation
!CUDA_VISIBLE_DEVICES=0 fairseq-generate data-bin/iwslt14.tokenized.de-en --path "{SAVE}/checkpoint_last10_avg.pt" --batch-size 128 --beam 4 --remove-bpe --lenpen 1 --gen-subset test --quiet 

Namespace(checkpoint_upper_bound=None, inputs=['/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt'], num_epoch_checkpoints=10, num_update_checkpoints=None, output='/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint_last10_avg.pt')
averaging checkpoints:  ['/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint70.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint69.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint68.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint67.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint66.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint65.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint64.pt', '/content/gdrive/My Drive/colab/payless/save_base/dynamic_conv_iwslt/checkpoint63.pt', '/

In [7]:
# Evaluation
!CUDA_VISIBLE_DEVICES=0 fairseq-generate data-bin/iwslt14.tokenized.de-en --path "{SAVE}/checkpoint_best.pt" --batch-size 128 --beam 4 --remove-bpe --lenpen 1 --gen-subset test --quiet 

Namespace(beam=4, bpe=None, cpu=False, criterion='cross_entropy', data='data-bin/iwslt14.tokenized.de-en', dataset_impl=None, decoding_format=None, diverse_beam_groups=-1, diverse_beam_strength=0.5, empty_cache_freq=0, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', iter_decode_eos_penalty=0.0, iter_decode_force_max_iter=False, iter_decode_max_iter=10, lazy_load=False, left_pad_source='True', left_pad_target='False', lenpen=1.0, load_alignments=False, log_format=None, log_interval=1000, lr_scheduler='fixed', lr_shrink=0.1, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, memory_efficient_fp16=False, min_len=1, min_loss_scale=0.0001, model_overrides='{}', momentum=0.99, nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, no_repeat_ngram_size=0, num_shards=1, num_workers=1, optimizer='nag', path='/cont